### Create camera

In [1]:
# Full reset of the camera
!echo 'jetson' | sudo -S systemctl restart nvargus-daemon && printf '\n'

from jetcam.csi_camera import CSICamera

camera = CSICamera(width=640, height=480, capthur_fps=30)

camera.running = False

[sudo] password for jetson: 
GST_ARGUS: Creating output stream
CONSUMER: Waiting until producer is connected...
GST_ARGUS: Available Sensor modes :
GST_ARGUS: 3280 x 2464 FR = 21.000000 fps Duration = 47619048 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 3280 x 1848 FR = 28.000001 fps Duration = 35714284 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1920 x 1080 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1640 x 1232 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1280 x 720 FR = 59.999999 fps Duration = 16666667 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: Running with following settings:
   Camera index = 0 
   Camera mode  = 4 
  

[ WARN:0@0.637] global cap_gstreamer.cpp:1728 open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


### Import libraries

In [2]:
import cv2
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget
from robot.jetracer import JetRacer
import numpy as np
import time

### Create camera preview

In [3]:
camera_widget = ipywidgets.Image(width=camera.width, height=camera.height)

display(camera_widget)

Image(value=b'', height='480', width='640')

### Create robot control

In [4]:
car = JetRacer(bus=7, signal_freq=50, servo_channel=0, motor_channel=1)

In [5]:
car.steering = 0.0
car.throttle = 0

In [ ]:
car.throttle = -0.18

display_cnt = 0
throttle_cnt = 0

blue_line_passed = 0
last_passed_blue_line_time = 0

while True:
    image = camera.read()
    edges = cv2.Canny(image,100,200)

    left_edge_y_pos = -1
    right_edge_y_pos = -1
    canny_left_edge_pos = -1
    canny_right_edge_pos = -1
    for y in range(460):
        y = 479-y
        left_color = np.sum(image[y-5:y, 5:10])
        right_color = np.sum(image[y-5:y, 630:635])
        if left_edge_y_pos == -1 and left_color < 4000:
            left_edge_y_pos = y
            image[y-5:y, 5:10] = 255
        if right_edge_y_pos == -1 and right_color < 4000:
            right_edge_y_pos = y
            image[y-5:y, 630:635] = 255
        if y > 290 and canny_left_edge_pos == -1 and edges[y,5] == 255:
            canny_left_edge_pos = y
            image[y-5:y, 0:640, 1] = 255
        if y > 290 and canny_right_edge_pos == -1 and edges[y,635] == 255:
            canny_rightt_edge_pos = y
            image[y-5:y, 0:640, 0] = 255
        if y > 120 and left_edge_y_pos != -1 and right_edge_y_pos != -1:
            break
    if canny_left_edge_pos > left_edge_y_pos or canny_right_edge_pos > right_edge_y_pos:
        if time.time()-last_passed_blue_line_time > 2:
            last_passed_blue_line_time = time.time()
            blue_line_passed += 1
            print(blue_line_passed)
    
    car.steering = -(left_edge_y_pos - right_edge_y_pos) / 420

    if blue_line_passed < 12 and throttle_cnt % 1 == 0:
        car.throttle = -0.18
    else:
        car.throttle = 0
        car.throttle = 0
    throttle_cnt += 1
    # if display_cnt % 10 == 0:
    #     camera_widget.value = bgr8_to_jpeg(image)
    # display_cnt += 1
    camera_widget.value = bgr8_to_jpeg(edges)